# Comparação entre Delta Lake e Apache Iceberg

Este notebook apresenta uma comparação entre Delta Lake e Apache Iceberg, destacando suas semelhanças, diferenças e casos de uso específicos.

## Configuração do Ambiente

Primeiro, vamos configurar o ambiente e criar uma sessão Spark com suporte a ambas as tecnologias.

In [ ]:
# Importar as bibliotecas necessárias
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

# Adicionar o diretório src ao PYTHONPATH
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(project_dir, 'src'))

# Importar os módulos do projeto
from spark_delta_iceberg.spark_session import create_spark_session
from spark_delta_iceberg.delta_operations import DeltaLakeOperations
from spark_delta_iceberg.iceberg_operations import IcebergOperations
from spark_delta_iceberg.sample_data import load_public_dataset

# Criar a sessão Spark
spark = create_spark_session("DeltaIcebergComparison")

# Criar instâncias das classes de operações
delta_ops = DeltaLakeOperations(spark)
iceberg_ops = IcebergOperations(spark)

print("Ambiente configurado com sucesso!")
print(f"Versão do Spark: {spark.version}")

## Carregamento de Dados Públicos

Vamos carregar um conjunto de dados públicos para usar em nossa comparação.

In [ ]:
# Carregar o dataset público
df_vendas, dataset_description = load_public_dataset(spark)

# Exibir a descrição do dataset
# print(dataset_description)

# Exibir o esquema
print("\nEsquema do DataFrame:")
df_vendas.printSchema()

# Exibir uma amostra dos dados
print("\nAmostra dos dados:")
df_vendas.show(5)

## Modelo ER e Estrutura de Dados

Vamos definir o modelo ER para nossos dados de vendas de supermercado.

### Modelo ER

```
+----------------+        +----------------+        +----------------+
|    Cliente     |        |     Venda      |        |    Produto     |
+----------------+        +----------------+        +----------------+
| id_cliente (PK)|<-----> | id_venda (PK)  | <----->| id_produto (PK)|
| nome           |        | data           |        | nome           |
| genero         |        | id_cliente (FK)|        | categoria      |
| tipo_cliente   |        | id_produto (FK)|        | preco_unitario |
| cidade         |        | quantidade     |        | custo          |
+----------------+        | preco_total    |        +----------------+
                          | pagamento      |
                          | avaliacao      |
                          +----------------+
```

In [ ]:
# Definir os caminhos e nomes das tabelas
delta_path = os.path.join(project_dir, "spark-warehouse", "vendas_supermercado_delta")
iceberg_db = "default"
iceberg_table = "vendas_supermercado_iceberg"

# Criar a tabela Delta Lake
delta_ops.create_table(
    df=df_vendas,
    table_path=delta_path,
    mode="overwrite",
    partition_by=["Payment"]
)

# Criar a tabela Iceberg
df_vendas.createOrReplaceTempView("temp_view")
iceberg_ops.create_table(
    df=df_vendas,
    table_name=iceberg_table,
    database=iceberg_db,
    partition_by=["Payment"]
)

print("Tabelas criadas com sucesso!")

## Comparação de Desempenho: INSERT

Vamos comparar o desempenho das operações INSERT entre Delta Lake e Apache Iceberg.

In [ ]:
import time

# Criar novos dados para inserção
# Vamos duplicar os dados existentes e modificar alguns valores
df_novos = df_vendas.limit(100).withColumn("Invoice ID", lit("New-") + col("Invoice ID"))

# Medir o tempo para inserção no Delta Lake
start_time = time.time()
delta_ops.insert_data(
    table_path=delta_path,
    new_data=df_novos,
    mode="append"
)
delta_insert_time = time.time() - start_time
print(f"Tempo de inserção no Delta Lake: {delta_insert_time:.2f} segundos")

# Medir o tempo para inserção no Iceberg
start_time = time.time()
iceberg_ops.insert_data(
    df=df_novos,
    table_name=iceberg_table,
    database=iceberg_db
)
iceberg_insert_time = time.time() - start_time
print(f"Tempo de inserção no Apache Iceberg: {iceberg_insert_time:.2f} segundos")

## Comparação de Desempenho: UPDATE

Vamos comparar o desempenho das operações UPDATE entre Delta Lake e Apache Iceberg.

In [ ]:
# Medir o tempo para atualização no Delta Lake
start_time = time.time()
delta_ops.update_data(
    table_path=delta_path,
    condition="`Product line` = 'Health and beauty'",
    update_expr={"Rating": "Rating + 0.5"}
)
delta_update_time = time.time() - start_time
print(f"Tempo de atualização no Delta Lake: {delta_update_time:.2f} segundos")

# Medir o tempo para atualização no Iceberg
start_time = time.time()
iceberg_ops.update_data(
    table_name=iceberg_table,
    database=iceberg_db,
    condition="`Product line` = 'Health and beauty'",
    update_expr={"Rating": "Rating + 0.5"}
)
iceberg_update_time = time.time() - start_time
print(f"Tempo de atualização no Apache Iceberg: {iceberg_update_time:.2f} segundos")

## Comparação de Desempenho: DELETE

Vamos comparar o desempenho das operações DELETE entre Delta Lake e Apache Iceberg.

In [ ]:
# Medir o tempo para exclusão no Delta Lake
start_time = time.time()
delta_ops.delete_data(
    table_path=delta_path,
    condition="`Product line` = 'Electronic accessories' AND Quantity < 3"
)
delta_delete_time = time.time() - start_time
print(f"Tempo de exclusão no Delta Lake: {delta_delete_time:.2f} segundos")

# Medir o tempo para exclusão no Iceberg
start_time = time.time()
iceberg_ops.delete_data(
    table_name=iceberg_table,
    database=iceberg_db,
    condition="`Product line` = 'Electronic accessories' AND Quantity < 3"
)
iceberg_delete_time = time.time() - start_time
print(f"Tempo de exclusão no Apache Iceberg: {iceberg_delete_time:.2f} segundos")

## Resumo da Comparação

Vamos resumir os resultados da comparação entre Delta Lake e Apache Iceberg.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dados para o gráfico
operations = ['INSERT', 'UPDATE', 'DELETE']
delta_times = [delta_insert_time, delta_update_time, delta_delete_time]
iceberg_times = [iceberg_insert_time, iceberg_update_time, iceberg_delete_time]

# Criar o gráfico de barras
x = np.arange(len(operations))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, delta_times, width, label='Delta Lake')
rects2 = ax.bar(x + width/2, iceberg_times, width, label='Apache Iceberg')

# Adicionar rótulos e título
ax.set_ylabel('Tempo (segundos)')
ax.set_title('Comparação de Desempenho: Delta Lake vs Apache Iceberg')
ax.set_xticks(x)
ax.set_xticklabels(operations)
ax.legend()

# Adicionar valores nas barras
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
plt.show()

## Comparação de Recursos

### Semelhanças entre Delta Lake e Apache Iceberg

1. **Transações ACID**: Ambos oferecem suporte a transações ACID para garantir a consistência dos dados.
2. **Time Travel**: Ambos permitem acessar versões anteriores dos dados.
3. **Operações DML**: Ambos suportam operações como INSERT, UPDATE, DELETE e MERGE.
4. **Evolução de Esquema**: Ambos permitem alterar o esquema dos dados sem afetar os consumidores.
5. **Formato de Arquivo Base**: Ambos usam o formato Parquet como formato de arquivo base.

### Diferenças entre Delta Lake e Apache Iceberg

| Característica | Delta Lake | Apache Iceberg |
|---------------|------------|----------------|
| **Origem** | Desenvolvido pela Databricks | Projeto da Apache Software Foundation |
| **Metadados** | Armazenados em arquivos JSON | Armazenados em arquivos Avro |
| **Integração** | Melhor integração com Databricks | Melhor integração com ecossistema Apache |
| **Particionamento** | Particionamento explícito | Particionamento oculto |
| **Compactação** | Suporte nativo a compactação de arquivos pequenos | Requer configuração adicional |
| **Streaming** | Suporte nativo a streaming com Structured Streaming | Suporte limitado a streaming |
| **Comunidade** | Comunidade menor, mais focada em Databricks | Comunidade maior e mais diversificada |

### Casos de Uso Recomendados

**Delta Lake é mais adequado para:**
- Ambientes Databricks
- Casos de uso com streaming em tempo real
- Quando a compactação automática de arquivos pequenos é importante
- Quando a integração com o ecossistema Databricks é necessária

**Apache Iceberg é mais adequado para:**
- Ambientes multi-engine (Spark, Flink, Presto, etc.)
- Quando a independência de fornecedor é importante
- Quando o particionamento oculto é desejado
- Quando a integração com o ecossistema Apache é necessária

## Conclusão

Neste notebook, comparamos o Delta Lake e o Apache Iceberg em termos de desempenho e recursos. Ambas as tecnologias oferecem recursos semelhantes para gerenciamento de dados em data lakes, como transações ACID, time travel e operações DML.

A escolha entre Delta Lake e Apache Iceberg depende principalmente do ecossistema em que você está trabalhando e dos requisitos específicos do seu caso de uso. Se você está trabalhando principalmente com Databricks, o Delta Lake pode ser a escolha mais natural. Se você precisa de interoperabilidade entre diferentes motores de processamento, o Apache Iceberg pode ser mais adequado.

Em termos de desempenho, ambas as tecnologias são comparáveis, com pequenas diferenças dependendo da operação específica e do tamanho dos dados. É recomendável realizar testes de desempenho com seus próprios dados e casos de uso para determinar qual tecnologia é mais adequada para suas necessidades. Caso você tenha lido até aqui, parabéns!